# Clone Forge

In [ ]:
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge.git

# Install Requirements

In [ ]:
%cd /kaggle/working/stable-diffusion-webui-forge
!pip install -r /kaggle/working/stable-diffusion-webui-forge/requirements_versions.txt
!pip install transformers==4.44.0
!pip install diffusers==0.29.2
!pip install peft==0.12.0
!pip install huggingface-hub==0.24.6

# Run With Ngrok
Note!

In [ ]:
# --- Variables ---

Ngrok_token = "2kWHhXogC048pvyVcoMwpunOSyt_siao5DXRai9jek1tgH3Q"  # Update with your ngrok token
Ngrok_domain = ""  # optional, leave empty if you don't have a domain
port = 7860
HOME_FOLDER = "/kaggle"

# -----------------

!pip install pyngrok==6.1.0  # Downgrade to a version that supports ngrok tunneling

from pyngrok import ngrok, conf
import gc

gc.collect()

if Ngrok_token:
    try:
        ngrok.set_auth_token(Ngrok_token)
        ngrok.kill()
        srv = ngrok.connect(port)
        print(f"Ngrok Tunnel is active at: {srv.public_url}")
        
        # Command to start
        !echo -e "A\nN" | python /kaggle/working/stable-diffusion-webui-forge/launch.py
        
    except Exception as e:
        print(f"Error starting ngrok tunnel: {e}")
else:
    print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the Ngrok_token field.')

# Install Model

### Install Checkpoints

In [ ]:
# Install a model to permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
# You can find other model and download it here just chage name and model url
model_url = "https://huggingface.co/lllyasviel/flux1-dev-bnb-nf4/resolve/main/flux1-dev-bnb-nf4-v2.safetensors?download=true"
model_name = "flux1-dev-bnb-nf4-v2.safetensors"

%cd /kaggle/working/stable-diffusion-webui-forge/models/Stable-diffusion
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

### Install *Lora*



In [ ]:
import os

# Set the model URL and name
model_url = "https://civitai.com/api/download/models/1026423?type=Model&format=SafeTensor"
model_name = "UltraRealPhoto.safetensors"

# Define the target directory
target_dir = "/kaggle/working/stable-diffusion-webui-forge/models/Lora"

# Create the directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Change to the target directory
%cd $target_dir

# Download the model file
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')


# Run with Localtunnel

In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

!python /kaggle/working/stable-diffusion-webui-forge/launch.py

# Launch with Piggy

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/stable-diffusion-webui-forge/launch.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:7860 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()